In [16]:
from dotenv import load_dotenv
import os
load_dotenv("local.env")

# https://thenewstack.io/getting-started-with-python-and-influxdb/
from influxdb import InfluxDBClient

user = os.getenv("INFLUX_USER")
password = os.getenv("INFLUX_PASS")

dbname = os.getenv("INFLUX_DBNAME")

client = InfluxDBClient(
    host="iotdash.austinvoicedata.com",
    path="influxdb:8086",
    port=443,
    ssl=True,
    verify_ssl=True,
    username=user,
    password=password,
    database=dbname
)

# query = 'SELECT * FROM radiobridge WHERE time > now() - 1h'
# query = 'select * from radiobridge'

query_where = 'select * from radiobridge where dev_eui=$deveui'
bind_params = {'deveui': '0080000004017e61'}
query = 'SELECT * FROM radiobridge LIMIT 4'

result = client.query(query)
# print("Result:\n{0}".format(result))
points = list(result.get_points())
print("Points:\n{0}".format(points))
# data = []
# meas="radiobridge2"

Points:
[{'time': '2022-01-24T21:26:45.648000Z', 'bandwidth': 125000.0, 'battery_voltage': None, 'dev_eui': '0080000004017e61', 'f_count': 0.0, 'frame_size': None, 'frequency': 905.1, 'gateway_eui': '647fda011d9e0000', 'gw_alt': None, 'gw_latitude': None, 'gw_longitude': None, 'humidity': None, 'message_type': 'Reset', 'packet_counter': 0.0, 'rcv_time': 1643059605.6475573, 'rssi': -110.0, 'snr': 7.8, 'spreading_factor': 9.0, 'temperature': None}, {'time': '2022-01-24T21:27:00.058000Z', 'bandwidth': 125000.0, 'battery_voltage': None, 'dev_eui': '0080000004017e61', 'f_count': 1.0, 'frame_size': None, 'frequency': 912.9, 'gateway_eui': '647fda011d9e0000', 'gw_alt': None, 'gw_latitude': None, 'gw_longitude': None, 'humidity': None, 'message_type': 'Voltage Sensor', 'packet_counter': 3.0, 'rcv_time': 1643059620.0584414, 'rssi': -109.0, 'snr': 5.5, 'spreading_factor': 10.0, 'temperature': None}, {'time': '2022-01-25T01:57:31.174339Z', 'bandwidth': 125000.0, 'battery_voltage': 3.2, 'dev_eui':

In [ ]:


# for p in points:
#     data.append("{meas},dev_eui={dev_eui},gateway_eui={gateway_eui} \
#         bandwidth={bandwidth},battery_voltage={battery_voltage},f_count={f_count}i {timestamp}"
#         .format(meas=meas,
#             dev_eui=p['dev_eui'],
#             gateway_eui=p['gateway_eui'],
#             bandwidth=int(p['bandwidth']),
#             battery_voltage=p['battery_voltage'],
#             f_count=int(p['f_count']),
#             timestamp=p['time'])
#     )
# print(F'Lines:\n{data}')

In [24]:
#print("Points:\n{0}".format(points))
data = []
meas="radiobridge2"

for p in points:
    print(p)
    linehead = F'{meas},dev_eui={p["dev_eui"]},gateway_eui={p["gateway_eui"]}'
    metrics = F'bandwidth={int(p["bandwidth"])}i,' + F'frequency={round(p["frequency"],1)},' + F'spreading_factor={int(p["spreading_factor"])}i,' \
                + F'f_count={int(p["f_count"])}i,' + F'rssi={int(p["rssi"])},' + F'snr={round(p["snr"],1)}'
    if 'frame_size' in p and p['frame_size'] is not None:
        metrics = metrics + F',frame_size={int(p["frame_size"])}i'
    if 'battery_voltage' in p and p['battery_voltage'] is not None:
        metrics = metrics + F',battery_voltage={p["battery_voltage"]}'
    if 'gw_alt' in p and p['gw_alt'] is not None:
        metrics = metrics + F',gw_alt={int(p["gw_alt"])}i'
    if 'gw_latitude' in p and p['gw_latitude'] is not None:
        metrics = metrics + F',gw_latitude={round(p["gw_latitude"],5)}'
    if 'gw_longitude' in p and p['gw_longitude'] is not None:
        metrics = metrics + F',gw_longitude={round(p["gw_longitude"],5)}'
    if 'temperature' in p and p['temperature'] is not None:
        metrics = metrics + F',temperature={round(p["temperature"],1)}'
    if 'humidity' in p and p['humidity'] is not None:
        metrics = metrics + F',humidity={round(p["humidity"],1)}'
    if 'message_type' in p and p['message_type'] is not None:
        metrics = metrics + F',message_type={p["message_type"]}'

    data.append(linehead + " " + metrics + " " + p['time'])

print(F'Lines:\n{data}')

{'time': '2022-01-24T21:26:45.648000Z', 'bandwidth': 125000.0, 'battery_voltage': None, 'dev_eui': '0080000004017e61', 'f_count': 0.0, 'frame_size': None, 'frequency': 905.1, 'gateway_eui': '647fda011d9e0000', 'gw_alt': None, 'gw_latitude': None, 'gw_longitude': None, 'humidity': None, 'message_type': 'Reset', 'packet_counter': 0.0, 'rcv_time': 1643059605.6475573, 'rssi': -110.0, 'snr': 7.8, 'spreading_factor': 9.0, 'temperature': None}
{'time': '2022-01-24T21:27:00.058000Z', 'bandwidth': 125000.0, 'battery_voltage': None, 'dev_eui': '0080000004017e61', 'f_count': 1.0, 'frame_size': None, 'frequency': 912.9, 'gateway_eui': '647fda011d9e0000', 'gw_alt': None, 'gw_latitude': None, 'gw_longitude': None, 'humidity': None, 'message_type': 'Voltage Sensor', 'packet_counter': 3.0, 'rcv_time': 1643059620.0584414, 'rssi': -109.0, 'snr': 5.5, 'spreading_factor': 10.0, 'temperature': None}
{'time': '2022-01-25T01:57:31.174339Z', 'bandwidth': 125000.0, 'battery_voltage': 3.2, 'dev_eui': '008000000

In [ ]:
print("1"+"2")